In [ ]:
import cantera as ct
import matplotlib.pyplot as plt
import numpy as np
import rmgpy.constants
import adsorbate_thermo
import rmgpy.species

In [ ]:
gas1 = ct.Solution('ptcombust.cti')
surf1 = ct.Interface('ptcombust.cti', 'Pt_surf', [gas1])

In [ ]:
surf1.species()

In [ ]:
gas1.TPX = 300, 101325, {'H2O': 1.0}

In [ ]:
# surf1.TPX = 300, 101325, {'H2O(S)': 0.1, 'PT(S)': 0.9}
surf1.TPX = 300, 101325, {'H2O(S)': 1.0}

In [ ]:
surf1.species()[2].thermo.coeffs
midpoint = surf1.species()[2].thermo.coeffs[0]
a_high = surf1.species()[2].thermo.coeffs[1:8]
a_low = surf1.species()[2].thermo.coeffs[8:]

In [ ]:
len(a_high)

In [ ]:
len(a_low)

In [ ]:
def get_thermo_from_NASA2(temperatures, a_low, a_high):
    # compute thermo properties from nasa polynomials
    T_switch = 1000.0
    R = rmgpy.constants.R
    i_switch = -1
    for i in range(len(temperatures)):
        if temperatures[i] == T_switch:
            i_switch = i

    cp_fit = np.zeros(len(temperatures))
    h_fit = np.zeros(len(temperatures))
    s_fit = np.zeros(len(temperatures))
    for (i, temp) in enumerate(temperatures):
        if temp <= T_switch:
            cp_fit[i] = a_low[0] + a_low[1] * temp + a_low[2] * temp**2.0 + a_low[3] * temp**3.0 + a_low[4] * temp**4.0
            h_fit[i] = a_low[0] * temp + a_low[1] / 2.0 * temp**2.0 + a_low[2] / 3.0 * temp**3.0 + a_low[3] / 4.0 * temp**4.0 + a_low[4] / 5.0 * temp**5.0 + a_low[5]
            s_fit[i] = a_low[0] * np.log(temp) + a_low[1] * temp + a_low[2] / 2.0 * temp**2.0 + a_low[3] / 3.0 * temp**3.0 + a_low[4] / 4.0 * temp**4.0 + a_low[6]
        else:
            cp_fit[i] = a_high[0] + a_high[1] * temp + a_high[2] * temp**2.0 + a_high[3] * temp**3.0 + a_high[4] * temp**4.0
            h_fit[i] = a_high[0] * temp + a_high[1] / 2.0 * temp**2.0 + a_high[2] / 3.0 * temp**3.0 + a_high[3] / 4.0 * temp**4.0 + a_high[4] / 5.0 * temp**5.0 + a_high[5]
            s_fit[i] = a_high[0] * np.log(temp) + a_high[1] * temp + a_high[2] / 2.0 * temp**2.0 + a_high[3] / 3.0 * temp**3.0 + a_high[4] / 4.0 * temp**4.0 + a_high[6]

    cp_fit *= R
    h_fit *= R
    s_fit *= R

    return cp_fit, h_fit, s_fit


In [ ]:
molecular_weight = 18.02
frequencies = [49.5, 68.6, 73.6, 102.0, 437.6, 452.9, 1596.3, 3675.6, 3787.0]
composition = {'H': 2, 'O': 1, 'C': 0, 'N': 0}
heat_of_formation_0K = -259.05  # kJ/mol
my_calc = adsorbate_thermo.AdsorbateThermoCalc(molecular_weight, frequencies, composition, heat_of_formation_0K, twoD_gas=True)
a_low2, a_high2 = my_calc.get_thermo2()

In [ ]:
temperatures = np.linspace(300, 2000, 201)
cp_fit, h_fit, s_fit = get_thermo_from_NASA2(temperatures, a_low, a_high)
cp_fit2, h_fit2, s_fit2 = my_calc.get_thermo_from_NASA2(a_low, a_high)

In [ ]:
dir(surf1)

In [ ]:
H_3 = np.zeros(len(temperatures))
Cp_3 = np.zeros(len(temperatures))
S_3 = np.zeros(len(temperatures))
for i, T in enumerate(temperatures):
    surf1.TPX = T, 101325, {'H2O(S)': 1.0}
    Cp_3[i] = surf1.cp_mole/1000.0
    H_3[i] = surf1.enthalpy_mole/1000.0
    S_3[i] = surf1.entropy_mole/1000.0

In [ ]:
plt.plot(temperatures, cp_fit)
plt.plot(my_calc.temperatures, cp_fit2)
plt.plot(temperatures, Cp_3)
plt.legend(['Cantera', 'Katrin', 'all Cantera'])

In [ ]:
plt.plot(temperatures, s_fit)
plt.plot(my_calc.temperatures, s_fit2)
plt.plot(temperatures, S_3)
plt.legend(['Cantera', 'Katrin', 'all Cantera'])

In [ ]:
plt.plot(temperatures, h_fit)
plt.plot(my_calc.temperatures, h_fit2)
plt.plot(temperatures, H_3)
plt.legend(['Cantera', 'Katrin', 'all Cantera'])